In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os.path
from pandas_ods_reader import read_ods

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers

import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

%matplotlib inline

In [25]:
fighter_sheet = "raw_fighter_details"
fight_sheet = "raw_total_fight_data"

fighters_ods = "UFC-DATA-DELETE-LATER/raw_fighter_details.ods"
fights_ods = "UFC-DATA-DELETE-LATER/raw_total_fight_data.ods"

In [3]:
fights_data = read_ods(fights_ods, fight_sheet)
fighters_data = read_ods(fighters_ods, fighter_sheet)

In [33]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

In [4]:
fights_data['R_SIG_STR_pct'] = fights_data['R_SIG_STR_pct'].str.strip('%').astype(float)
fights_data['B_SIG_STR_pct'] = fights_data['B_SIG_STR_pct'].str.strip('%').astype(float)
fights_data['R_TD_pct'] = fights_data['R_TD_pct'].str.strip('%').astype(float)
fights_data['B_TD_pct'] = fights_data['B_TD_pct'].str.strip('%').astype(float)

In [5]:
fights_data['R_SIG_STR_pct'] = fights_data['R_SIG_STR_pct']/100
fights_data['B_SIG_STR_pct'] = fights_data['B_SIG_STR_pct']/100
fights_data['R_TD_pct'] = fights_data['R_TD_pct']/100
fights_data['B_TD_pct'] = fights_data['B_TD_pct']/100


In [34]:
newfights_data = fights_data.drop(["R_TOTAL_STR.", "B_TOTAL_STR.", "R_TD", "B_TD"], axis=1)
newfights_data.head()

,R_fighter,B_fighter,R_KD,B_KD,R_SIG_STR.,B_SIG_STR.,R_SIG_STR_pct,B_SIG_STR_pct,R_TD_pct,B_TD_pct,R_SUB_ATT,B_SUB_ATT,R_PASS,B_PASS,R_REV,B_REV,R_HEAD,B_HEAD,R_BODY,B_BODY,R_LEG,B_LEG,R_DISTANCE,B_DISTANCE,R_CLINCH,B_CLINCH,R_GROUND,B_GROUND,win_by,last_round,last_round_time,Format,Referee,date,location,Fight_type,Winner
0,Henry Cejudo,Marlon Moraes,0.0,0.0,90 of 171,57 of 119,0.52,0.47,0.25,0.00,1.0,0.0,1.0,0.0,0.0,0.0,73 of 150,35 of 89,13 of 16,7 of 8,4 of 5,15 of 22,45 of 118,54 of 116,19 of 23,2 of 2,26 of 30,1 of 1,KO/TKO,3.0,4:51,5 Rnd (5-5-5-5-5),Marc Goddard,"June 08, 2019","Chicago, Illinois, USA",UFC Bantamweight Title Bout,Henry Cejudo
1,Valentina Shevchenko,Jessica Eye,1.0,0.0,8 of 11,2 of 12,0.72,0.16,1.00,0.00,1.0,0.0,3.0,0.0,0.0,0.0,4 of 5,0 of 7,4 of 6,0 of 2,0 of 0,2 of 3,5 of 8,2 of 12,2 of 2,0 of 0,1 of 1,0 of 0,KO/TKO,2.0,0:26,5 Rnd (5-5-5-5-5),Robert Madrigal,"June 08, 2019","Chicago, Illinois, USA",UFC Women's Flyweight Title Bout,Valentina Shevchenko
2,Tony Ferguson,Donald Cerrone,0.0,0.0,104 of 200,68 of 185,0.52,0.36,0.00,1.00,0.0,0.0,0.0,0.0,0.0,0.0,65 of 144,43 of 152,25 of 37,15 of 23,14 of 19,10 of 10,103 of 198,68 of 184,1 of 2,0 of 1,0 of 0,0 of 0,TKO - Doctor's Stoppage,2.0,5:00,3 Rnd (5-5-5),Dan Miragliotta,"June 08, 2019","Chicago, Illinois, USA",Lightweight Bout,Tony Ferguson
3,Jimmie Rivera,Petr Yan,0.0,2.0,73 of 192,56 of 189,0.38,0.29,0.00,0.33,0.0,0.0,0.0,1.0,0.0,0.0,42 of 145,40 of 166,15 of 24,13 of 19,16 of 23,3 of 4,60 of 173,42 of 167,9 of 15,10 of 12,4 of 4,4 of 10,Decision - Unanimous,3.0,5:00,3 Rnd (5-5-5),Kevin MacDonald,"June 08, 2019","Chicago, Illinois, USA",Bantamweight Bout,Petr Yan
4,Tai Tuivasa,Blagoy Ivanov,0.0,1.0,64 of 144,73 of 123,0.44,0.59,0.00,1.00,0.0,0.0,0.0,0.0,0.0,0.0,39 of 114,65 of 114,6 of 7,7 of 8,19 of 23,1 of 1,50 of 126,62 of 111,14 of 18,5 of 6,0 of 0,6 of 6,Decision - Unanimous,3.0,5:00,3 Rnd (5-5-5),Dan Miragliotta,"June 08, 2019","Chicago, Illinois, USA",Heavyweight Bout,Blagoy Ivanov


In [14]:
temp_df1 = fights_data['R_TOTAL_STR.'].str.split(' of ', expand = True)
temp_df1['R_TOTAL_STR.'] = temp_df1[0].astype(float)/temp_df1[1].astype(float)

temp_df2 = fights_data['B_TOTAL_STR.'].str.split(' of ', expand = True)
temp_df2['B_TOTAL_STR.'] = temp_df2[0].astype(float)/temp_df2[1].astype(float)

temp_df3 = fights_data['R_TD'].str.split(' of ', expand = True)
temp_df3['R_TD'] = temp_df3[0].astype(float)/temp_df3[1].astype(float)

temp_df4 = fights_data['B_TD'].str.split(' of ', expand = True)
temp_df4['B_TD'] = temp_df3[0].astype(float)/temp_df3[1].astype(float)

In [15]:
newtemp_df1 = temp_df1.drop([0, 1], axis=1)
newtemp_df2 = temp_df2.drop([0, 1], axis=1)
newtemp_df3 = temp_df3.drop([0, 1], axis=1)
newtemp_df4 = temp_df4.drop([0, 1], axis=1)

In [35]:
df69 = newfights_data.join(newtemp_df1)
df70 = df69.join(newtemp_df2)
df71 = df70.join(newtemp_df3)
df72 = df71.join(newtemp_df4)

df72.head()


,R_fighter,B_fighter,R_KD,B_KD,R_SIG_STR.,B_SIG_STR.,R_SIG_STR_pct,B_SIG_STR_pct,R_TD_pct,B_TD_pct,R_SUB_ATT,B_SUB_ATT,R_PASS,B_PASS,R_REV,B_REV,R_HEAD,B_HEAD,R_BODY,B_BODY,R_LEG,B_LEG,R_DISTANCE,B_DISTANCE,R_CLINCH,B_CLINCH,R_GROUND,B_GROUND,win_by,last_round,last_round_time,Format,Referee,date,location,Fight_type,Winner,R_TOTAL_STR.,B_TOTAL_STR.,R_TD,B_TD
0,Henry Cejudo,Marlon Moraes,0.0,0.0,90 of 171,57 of 119,0.52,0.47,0.25,0.00,1.0,0.0,1.0,0.0,0.0,0.0,73 of 150,35 of 89,13 of 16,7 of 8,4 of 5,15 of 22,45 of 118,54 of 116,19 of 23,2 of 2,26 of 30,1 of 1,KO/TKO,3.0,4:51,5 Rnd (5-5-5-5-5),Marc Goddard,"June 08, 2019","Chicago, Illinois, USA",UFC Bantamweight Title Bout,Henry Cejudo,0.543956,0.487603,0.25,0.25
1,Valentina Shevchenko,Jessica Eye,1.0,0.0,8 of 11,2 of 12,0.72,0.16,1.00,0.00,1.0,0.0,3.0,0.0,0.0,0.0,4 of 5,0 of 7,4 of 6,0 of 2,0 of 0,2 of 3,5 of 8,2 of 12,2 of 2,0 of 0,1 of 1,0 of 0,KO/TKO,2.0,0:26,5 Rnd (5-5-5-5-5),Robert Madrigal,"June 08, 2019","Chicago, Illinois, USA",UFC Women's Flyweight Title Bout,Valentina Shevchenko,0.925000,0.807692,1.00,1.00
2,Tony Ferguson,Donald Cerrone,0.0,0.0,104 of 200,68 of 185,0.52,0.36,0.00,1.00,0.0,0.0,0.0,0.0,0.0,0.0,65 of 144,43 of 152,25 of 37,15 of 23,14 of 19,10 of 10,103 of 198,68 of 184,1 of 2,0 of 1,0 of 0,0 of 0,TKO - Doctor's Stoppage,2.0,5:00,3 Rnd (5-5-5),Dan Miragliotta,"June 08, 2019","Chicago, Illinois, USA",Lightweight Bout,Tony Ferguson,0.520000,0.367568,NaN,NaN
3,Jimmie Rivera,Petr Yan,0.0,2.0,73 of 192,56 of 189,0.38,0.29,0.00,0.33,0.0,0.0,0.0,1.0,0.0,0.0,42 of 145,40 of 166,15 of 24,13 of 19,16 of 23,3 of 4,60 of 173,42 of 167,9 of 15,10 of 12,4 of 4,4 of 10,Decision - Unanimous,3.0,5:00,3 Rnd (5-5-5),Kevin MacDonald,"June 08, 2019","Chicago, Illinois, USA",Bantamweight Bout,Petr Yan,0.389744,0.302083,0.00,0.00
4,Tai Tuivasa,Blagoy Ivanov,0.0,1.0,64 of 144,73 of 123,0.44,0.59,0.00,1.00,0.0,0.0,0.0,0.0,0.0,0.0,39 of 114,65 of 114,6 of 7,7 of 8,19 of 23,1 of 1,50 of 126,62 of 111,14 of 18,5 of 6,0 of 0,6 of 6,Decision - Unanimous,3.0,5:00,3 Rnd (5-5-5),Dan Miragliotta,"June 08, 2019","Chicago, Illinois, USA",Heavyweight Bout,Blagoy Ivanov,0.452055,0.618321,NaN,NaN


In [19]:
df72['new_column'] = np.where(df72['R_fighter'] == df72['Winner'], '0', '1')
df72.new_column.head()

0    0
1    0
2    0
3    1
4    1
Name: new_column, dtype: object

In [28]:
df72 = df72.dropna()

In [31]:
# Creating input feature
X = df72[['R_KD','B_KD','R_SIG_STR_pct', 'B_SIG_STR_pct', 'R_TOTAL_STR.', 'B_TOTAL_STR.',
                 'R_TD','B_TD','R_TD_pct','B_TD_pct','R_SUB_ATT','B_SUB_ATT','R_PASS','B_PASS','R_REV','B_REV']]

# Creating target variable
y = df72[['new_column']]

#Creating Train and Test dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#Normalizing the input features
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
X_train = sc_x.fit_transform(X_train)
X_test = sc_x.transform(X_test)

#Normalizing the target variable
sc_y = StandardScaler()
y_train = sc_y.fit_transform(y_train)
y_test = sc_y.transform(y_test)

# Creating the Linear regressor
regressor = LinearRegression()

# fitting the training data to the Linear Regressor
regressor.fit(X_train,y_train)

# Checking the model coefficients
regressor.coef_

array([[-0.11399278,  0.21414019, -0.02037709,  0.17547938, -0.20782453,
         0.00521984, -0.29519207, -0.29519207,  0.4851173 ,  0.05905357,
        -0.0876606 ,  0.03285165, -0.11330882,  0.20816895, -0.07294054,
         0.03974156]])